In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough,RunnableParallel
from langchain_community.document_loaders import TextLoader

In [2]:
ollama_llm=OllamaLLM(model="llama3")
parser=StrOutputParser()

In [3]:
loader=TextLoader('data.txt',encoding='utf-8')
document=loader.load()

In [13]:
document

[Document(metadata={'source': 'data.txt'}, page_content="{Question: What are the admission requirements for international students applying to Bachelor of Business programs at Southern Academy of Higher Education (SAHE)? Answer: Admission requirements typically include both language and academic requirements. Students will be required to provide copies of academic transcripts, English language proficiency test scores, a completed application form, and other documentation when applying for their chosen course.}{Question: What is Entry requirements? Answer: All applicants, whether domestic or international, must be at least 18 years old at the time of enrollment to be eligible for SAHE undergraduate programs. Domestic students need to meet one of the following minimum academic entry requirements to qualify for admission: they must have completed Year 12, achieving a minimum ATAR score of 60 or an equivalent qualification, or have successfully completed approved higher education studies s

In [17]:
spliter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
pages=spliter.split_documents(document)

In [18]:
pages

[Document(metadata={'source': 'data.txt'}, page_content='{Question: What are the admission requirements for international students applying to Bachelor of Business programs at Southern Academy of Higher Education (SAHE)? Answer: Admission requirements typically include both language and academic requirements. Students will be required to provide copies of academic transcripts, English language proficiency test scores, a completed application form, and other documentation when applying for their chosen course.}{Question: What is Entry requirements?'),
 Document(metadata={'source': 'data.txt'}, page_content='other documentation when applying for their chosen course.}{Question: What is Entry requirements? Answer: All applicants, whether domestic or international, must be at least 18 years old at the time of enrollment to be eligible for SAHE undergraduate programs. Domestic students need to meet one of the following minimum academic entry requirements to qualify for admission: they must h

In [19]:
vector_storage=FAISS.from_documents(pages,embedding=OllamaEmbeddings(model='llama3'))

In [20]:
retriever=vector_storage.as_retriever()

In [29]:
retriever.invoke('list public holidays of 2024')

[Document(metadata={'source': 'data.txt'}, page_content='exams will begin on Monday, 28 September 2026, with a replacement additional exam week on Monday, 5 October 2026. Semester marks are to be released on Thursday, 15 October 2026, and the semester break will span from Monday, 7 September 2026 to Sunday, 28 February 2027. Special permission may be sought for late acceptance or enrolment in exceptional cases. Public holidays for the years 2024, 2025, and 2026 include New Year’s Day on 1 January each year, Australia Day on 26 January 2024, 27'),
 Document(metadata={'source': 'data.txt'}, page_content='are made only if the Dean is confident that the applicant has a reasonable expectation of success based on their portfolio submission. The Dean is responsible for making admission decisions for all mature age applicants, ensuring that their application reflects a strong potential for success in the program based on the provided information.}{Question: What is the application deadline for

In [22]:
template=("""
You are AI-powered chatbot designed to provide information and assistance for customers based on the context provided to you only.
Context: {context}
Question: {question}
""")

In [9]:
prompt=ChatPromptTemplate.from_template(template)
prompt.format(context='context here',question='What does SAHE DO? ')

'Human: \nYou are AI-powered chatbot designed to provide information and assistance for customers based on the context provided to you only.\nContext: context here\nQuestion: What does SAHE DO? \n'

In [10]:
result=RunnableParallel(context=retriever,question=RunnablePassthrough())
chain=result|prompt|ollama_llm|parser

In [28]:
chain.invoke('list public holidays of 2024')

"According to the provided context, the public holidays for the year 2024 are:\n\n* King's Birthday on 10 June 2024\n* Labour Day on 7 October 2024\n* Christmas and Boxing Day on 25-26 December 2024"

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.document_loaders import TextLoader

# Initialize LLM and parser
ollama_llm = OllamaLLM(model="llama3")
parser = StrOutputParser()

# Load documents
loader = TextLoader('extracted_text.txt', encoding='utf-8')
document = loader.load()

# Split documents into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
pages = splitter.split_documents(document)

# Create FAISS vector store from documents
vector_storage = FAISS.from_documents(pages, embedding=OllamaEmbeddings(model='llama3'))
retriever = vector_storage.as_retriever()

# Define prompt template
template = """
You are an AI-powered chatbot designed to assist visitors of our university website. Your role is to provide accurate and helpful information based on the provided context.
Context: {context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Create the chain
result = RunnableParallel(context=retriever, question=RunnablePassthrough())
chain = result | prompt | ollama_llm | parser

# Example question
response = chain.invoke('Give me Fee Structure for the Business Program for international students')
print(response)


ConnectError: [WinError 10061] No connection could be made because the target machine actively refused it

In [20]:
response = chain.invoke('when did the university establish?')
print(response)


I'm happy to help! However, I couldn't find any information about the university's establishment date in the provided context. The documents you shared appear to be related to studying at SAHE, tuition protection services, and appeals procedures, but they don't mention the university's founding or establishment year. If you're looking for this information, I suggest checking the university's official website, contact their administration office, or consult with a staff member who may have access to such information.


In [2]:
!pip install PyMuPDF

   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 3.2/3.2 MB 27.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.2 MB ? eta -:--:--
   ----------------------- ---------------- 7.6/13.2 MB 36.2 MB/s eta 0:00:01
   ---------------------------------------- 13.2/13.2 MB 33.1 MB/s eta 0:00:00


In [3]:
import fitz  # PyMuPDF

# Load PDF file
pdf_document = "Student-Handbook-V1.0.pdf"
pdf = fitz.open(pdf_document)

# Extract text
text = ""
for page_num in range(len(pdf)):
    page = pdf.load_page(page_num)
    text += page.get_text()

pdf.close()

# Save extracted text to a file
with open("extracted_text.txt", "w", encoding="utf-8") as text_file:
    text_file.write(text)